### Đọc ghi tệp
- Đến nay, ta đã thảo luận về cách xử lý dữ liệu và cách xây dựng, huấn luyện, kiểm tra những mô hình học sâu. Tuy nhiên, có thể đến một lúc nào đó ta sẽ cảm thấy hài lòng với những gì thu được và muốn lưu lại kết quả để sau này sử dụng trong những bối cảnh khác nhau (thậm chí có thể đưa ra kết quả dựn đoán khi triển khai). 
- Ngoài ra, khi vận hành một quá trình huấn luyện dài hơi, tốt nhất là lưu kết quả trung gian một cách định kỳ nhằm đảm bảo rằng ta sẽ không mất kết quả tính toán sau nhiều ngày nến chẳng may gặp vấn đề.

#### 1. Đọc và lưu các ndarray
- Đối với ndarray riêng lẻ, ta có thể sử dụng trực tiếp các hàm load và save để đọc và ghi tương ứng. Cả hai hàm đều yêu cầu ta cung cấp tên, hàm save yêu cầu đầu vào với biến đã được lưu.

In [ ]:
from mxnet import np, npx
from mxnet.gluon import nn
npx.set_np()

x = np.arange(4)
npx.save('x-file', x)

Bây giờ, ta có thể đọc lại dữ liệu từ các tệp được bảo lưu vào trong bộ nhớ.

In [2]:
x2 = npx.load('x-file')
x2

[array([0., 1., 2., 3.])]

MXNet đồng thời cho phép ta lưu một danh sách các ndarray và đọc lại chúng vào trong bộ nhớ.

In [3]:
y = np.zeros(4)
npx.save('x-files', [x, y])
x2, y2 = npx.load('x-files')
x2, y2

(array([0., 1., 2., 3.]), array([0., 0., 0., 0.]))

Ta còn có thể ghi và đọc một từ điển ánh xạ từ một chuỗi sang một ndarray. Điều này khá là thuận tiện khi chúng ta muốn đọc hoặc ghi tất cả các trọng số của mô hình.

In [4]:
mydict = {'x': x, 'y' : y}
npx.save('mydict', mydict)
mydict2 = npx.load('mydict')
mydict2

{'x': array([0., 1., 2., 3.]), 'y': array([0., 0., 0., 0.])}

#### 2. Tham số mô hình Gluon
- Khả năng lưu từng vector trọng số đơn lẻ hoặc các ndarray tensor khác là hữu ích nhưng sẽ mất nhiều thời gian nếu ta muốn lưu và sau đó nạp lại toàn bộ mô hình do có thể có hàng trăm nhóm tham số rải rác xuyên suốt mô hình.
- Vì lý do đó Gluon cung cấp sẵn tính năng lưu và nạp toàn bộ các mạng. Một chi tiết quan trọng cần lưu ý là __chức năng này chỉ lưu các tham số của mô hình, không phải toàn bộ mô hình__. Tức là nếu ta có một MLP ba tầng, ta __cần chỉ rõ kiến trúc này một cách riêng lẻ__.  Lý do là vì bản thân các mô hình có thể chứa mã nguồn bất kỳ, chúng không được thêm vào tập tin dễ dàng như các tham số.
- Vì vậy, để khôi phục lại một mô hình thì chũng ta cần xây dựng lại kiến trúc của nó từ mã nguồn rồi nạp các tham số từ ổ cứng vào kiến trúc này.
- Việc khởi tạo trễ lúc này rất có lợi bởi ta chỉ cần định nghĩa một mô hình mà không cần gán giá trị cụ thể cho tham số.

In [5]:
class MLP(nn.Block):
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Dense(256, activation='relu')
        self.output = nn.Dense(10)
    def forward(self, x):
        return self.output(self.hidden(x))

net = MLP()
net.initialize()
x = np.random.uniform(size = (2, 10))
net(x)

array([[ 0.04769547,  0.01613292,  0.0163198 , -0.05325831, -0.01625958,
        -0.00756938, -0.0219499 ,  0.09401564, -0.02075765,  0.03281157],
       [ 0.05781221,  0.01893527, -0.00019526, -0.0217602 , -0.00246077,
        -0.04333879, -0.00157848,  0.08837229,  0.03380634,  0.02674502]])

Tiếp theo, ta lưu các tham số của mô hình vào tệp __mlp.params__. Những khối Gluon hỗ trợ phương thức từ hàm __save_parameter__ nhằm ghi lại tất cả các tham số vào ổ cứng được cung cấp với một chuỗi những tên tệp.

In [6]:
net.save_parameters('mlp.params')

Để khôi phục mô hình, ta tạo một đối tượng khác dựa trên mô hình MLP gốc. Thay vì khởi tạo ngẫu nhiên những tham số của mô hình, ta đọc các tham số được lưu trực tiếp trong tập tin sử dụng __load_parameters__.

In [7]:
clone = MLP()
clone.load_parameters('mlp.params')

In [8]:
yclone = clone(x)
y = net(x)
yclone == y

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])